In [346]:
%load_ext autoreload
%autoreload

import pandas as pd
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
import texcrapy
from ckonlpy.tag import Twitter, Postprocessor
import json
from soynlp.word import WordExtractor
from soynlp.tokenizer import MaxScoreTokenizer, LTokenizer
from soynlp.noun import LRNounExtractor_v2
import math
import os
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 아이템별 쿼리데이터(dict) 만들기

In [2]:
df = pd.read_excel('keywords and logos.xlsx', sheet_name='20190215')[['shortname','kw_supporter','kw_supported','keywords']]; df
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

qry_base = {row.shortname:_or(row.keywords) for row in df.itertuples()}
supporters = df.shortname[df.kw_supporter==True]
qry_sup = ' OR '.join([qry_base[sup] for sup in supporters]); qry_sup
qry = {row.shortname: '(' + qry_base[row.shortname] + ') AND (' + qry_sup + ')' if row.kw_supported==True else qry_base[row.shortname] for row in df.itertuples()}

In [3]:
qry

{'ootd': '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏',
 'category': '#designer OR #streetwear OR #luxury OR #casual OR #sportswear OR #디자이너 OR #길거리패션 OR #럭셔리 OR #캐주얼 OR #캐쥬얼 OR #스포츠웨어 OR #스포츠패션',
 'fashion': '#fashion OR #brand OR #fashionbrand OR #shoe OR #shoes OR #clothes OR #wear OR #apparel OR #패션 OR #브랜드 OR #패션브랜드 OR #옷 OR #의류 OR #슈즈 OR #신발',
 '8seconds': '#8seconds OR #8second OR #에잇세컨즈 OR #8세컨즈',
 'adidas': '#adidas OR #아디다스',
 'adidasoriginal': '#adidasoriginal OR #아디다스오리지널스 OR #아디다스오리지널 OR #아디다스오리지날스 OR #아디다스오리지날',
 'asics': '#asics OR #아식스',
 'balenciaga': '#balenciaga OR #balensiaga OR #발렌시아가',
 'bally': '#bally OR #발리',
 'beanpole': '#beanpole OR #빈폴',
 'benetton': '#benetton OR #베네통',
 'blackyak': '#blackyak OR #블랙야크',
 'buckaroo': '#buckaroo OR #버커루',
 'burberry': '#burberry OR #버버리',
 'calvinklein': '#calvinklein OR #케빈클라인 OR #켈빈클라인 OR #캘빈클라인',
 'canadagoose': '#canadagoose OR #캐나다구스',
 'chanel':

In [17]:
%%time
what = ['id', 'timestamp', 'text']
texcrapy.scrap(qry, what=what, lang='ko', end='2019-01-31', download_to='scrapped/twitter')

INFO: queries: ['#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-01-01 until:2010-06-15', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-06-15 until:2010-11-28', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2010-11-28 until:2011-05-13', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2011-05-13 until:2011-10-26', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2011-10-26 until:2012-04-09', '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏 since:2012-04-09 until:2012-09-22', '#dailylook OR #outfit OR #bestofstreetwear OR #bestst


Wall time: 5h 1min 17s


# 사전 만들기

In [143]:
twitter = Twitter()

In [144]:
newwords_kw = [w.strip() for w in ', '.join(df.keywords).split(',')]
newwords_custom = [
    '매거진','1박2일','아우터','항공점퍼','청치마','맨투맨','응답하라','추억팔이','레드벨벳','배꼽티','켄달제너','패션템',
    '스타일리쉬','스타일리시','나들이룩','섹시','섹시미','스타일링','입소문','비현실','세일','레스모아','요거','무개념',
    '보좌진','언제나','간신히','가성비','g1','10k','워킹화','쿠션감','후원금','기업','오니츠카','오니츠카타이거','농구화',
    '황금비율','대다나다','일본기업','슬립온','맥시코','데일리','착화감','마라톤화','젤라이트','너무','예뻐',
    '젤카야노','카야노','신박','이쁜','런닝화','러닝화','체리핑크','체리핑','퓨젝스','골프화','자세히','유투버','신상',
    '신상품','던롭','국대','단체복','스티브잡스','빅토리아시크릿','잡스타그램','a6','테스트모니얼','스트릿패션','스트릿브랜드',
    '굿즈','베리베리','반팔티','지급품','직영점','패셔니스타','패션피플','런스타그램','리미티드에디션','한동안','어글리','젤버즈',
    '아이스버킷챌린지','아이스버킷','챌린지','메탈릭','활력','대폭우','체열반사','보온성','포도몰','아까운','지나친','지티쿨',
    '2hchic','귀욥','클라이마웜','어디서나','어디서든','국가적','투박','젤버즈','키코코스타디노브','코스타디노브',
    'gelburz','월드컵경기장','스파이크리스','매력적','다르빗유','다르빗슈','스와로','반얀트리','그리고','신공항',
    '신발스타그램','선물스타그램','멍스타그램','개스타그램','다리스타그램','왼팔','오른팔','티샤스',
    '언제쯤','푸짐','교환권','서리풀','샌달','클락스','야구화','두류공원','피규어','열심히','브리온',
    '등산화','하그로프스','트레킹화','확실히','멍미','안전화','발성형','아식플','기능성',
    '한국소비자원','흡수성','건조성','오니쓰카',
    '변별력','열정적','혁신적','애슬레저','융합의다리','카이스트교','수침교','라이트쇼','종합운동장','젤소노마',
    '가장교','신스틸러','당일배송','번호이동','꾸리꾸리','한정수량',
    '노티카','키네스','64g','헤어밴드','키보드랙','앙큼','균일가','악세서리','구스다운',
    '친구분','일탈','깔맞춤','축스타그램','코파문디알','젝퍼셀','멕시코66','맥시코66','mexico66',
    '식탐송','리마커블트웬티즈','인센티브','맞이','텅장','지이크파렌하이트','화보집','모닝런',
    '무지개런','노익스큐즈','나스타그램','에프터샥','아미노바이탈','차크라','메달고',
    '진흥공단','스피돔','토리브엔터테인먼트','디자드','엔도서','듸제이불화자씨','베라','생일빵','싸인북',
    '귀여운','플리즈','적당히','물개박수','원아페','노박조코비치','당첨자','일본스니커','도쿄도민',
    '후디티셔츠','레이업로','죄책감런','에너제틱','새로움','그이상','아식스코리아','수촌교'
    '인스파이어','주행거리','마이다스','콀레','오리지날','한표','퓨젝스러쉬','달리자','스프라이핏','게이밍',
    '운동스타그램','만보','참가','열시미','자연추남','미션완료','고마워',
    '젤스팟라이트','운동하는','질러','충전재','디테일','인터벌','gel-ace','무료배송', 
    
]
newwords = list(set(newwords_kw + newwords_custom))
twitter.add_dictionary(newwords, 'Noun')

In [151]:
stopdoc = {'아베골프','포도몰','퐁당','[event]'}

stoptags = {'Foreign','Punctuation','KoreanParticle','Josa','Eomi','PreEomi','Exclamation','Determiner'}
passtags = set(twitter.tagset.keys()) - stoptags
stopwords = {'은','는','이','가','그','후','를','로','주','딱','오','아','다','내','껄','란','시',
             '데','돼','난','이나','거나','좀','들','중','기','년','거','까지','및','된'
             'i','ve','a','got','x','jpg','v','m','s','p','님','님들','구나','와','까','넹','u','is',
             '헐','f','w','의','d','습','니당','있','어요','be','and','pic','댁','욧','너','어','in','o'
            }
ngrams = [
    ('the','boyz'),
    ('어글리','슈즈'),
    ('리미티드','에디션'),
    ('키코','코스타디노브'), ('kiko','kostadinov'),
    ('젤','카야노'),
    ('젤','라이트'),('gel','lyte'),
    ('스튜어트','와이츠먼'),
    ('주니','블랙'),
    ('jun','hyo','seong'),
    ('gel','ptg'),
    ('오니츠카','타이거'),
    ('노박','조코비치'),
    ('후디','티셔츠'),
    ('최종병기','활'),
    ('아이제아','토마스'),
    ('gel','ace')
]

In [152]:
tagger = Postprocessor(twitter, stopwords=stopwords, passtags=passtags, ngrams=ngrams)

### text = 트위터 글 하나하나
### doc = {text: ~, ...}
### docs = [doc1, doc2, ...]

In [153]:
with open('scrapped/twitter/asics (2010.01.01-2019.01.31).json', encoding='UTF-8-sig') as f:
    jsn = json.load(f)
    
%time toks = texcrapy.tokenize_docs(*jsn['asics'], with_pos=True, tagger=tagger, stopdoc=stopdoc)
toks

Wall time: 1.31 s


[([('족적', 'Noun'), ('asics', 'Noun')], '족적.jpg asics'),
 ([('박근혜', 'Noun'),
   ('깜짝', 'Noun'),
   ('시구', 'Noun'),
   ('글러브', 'Noun'),
   ('태극마크', 'Noun'),
   ('운동화', 'Noun'),
   ('일제', 'Noun'),
   ('아식스', 'Noun'),
   ('이투데이', 'Noun'),
   ('아베', 'Noun'),
   ('오빠', 'Noun'),
   ('날좀', 'Noun'),
   ('보소', 'Verb'),
   ('날좀', 'Noun'),
   ('보소', 'Verb')],
  '박근혜 깜짝 시구...글러브엔 태극마크ㆍ운동화는 일제 아식스 ? 이투데이 에서 아베오빠 날좀보소 날좀보소'),
 ([('박근혜', 'Noun'),
   ('깜짝', 'Noun'),
   ('시구', 'Noun'),
   ('글러브', 'Noun'),
   ('태극마크', 'Noun'),
   ('운동화', 'Noun'),
   ('일제', 'Noun'),
   ('아식스', 'Noun'),
   ('이투데이', 'Noun'),
   ('이럴', 'Adjective'),
   ('줄', 'Noun'),
   ('알았다', 'Verb'),
   ('팬티', 'Noun'),
   ('욱일승천기', 'Noun'),
   ('들었을', 'Verb')],
  '박근혜 깜짝 시구...글러브엔 태극마크ㆍ운동화는 일제 아식스 ? 이투데이 에서 내 이럴 줄 알았다 팬티에는 욱일승천기 들었을껄'),
 ([('무개념', 'Noun'),
   ('여인', 'Noun'),
   ('국산', 'Noun'),
   ('운동화', 'Noun'),
   ('신기', 'Noun'),
   ('바', 'Noun'),
   ('것', 'Noun'),
   ('정말', 'Noun'),
   ('지나친', 'Noun'),
   ('기대', 'Noun'),
   ('잖아요', 'Ve

In [462]:
newwords_custom = {
    '이벤트','스포츠','이투데이','보좌진','홍보수석','체리핑','기업','오니츠카','오니츠카타이거',
    '마일드세븐','세븐일레븐','1949년','오니츠카기하치로'
}

stopwords = {
    'vi','a','is','ve',
    '은','는','이','가','에서','시','엔','등','내','줄','데','라는','야','를','을','지','이나','로',
    '으로','나','있는','수','요','하고','에','년','생'
}

In [463]:
scores = {noun:nounscore.score for noun, nounscore in nouns.items()}

for w in newwords_kw:
    scores[w] = 1
    
for w in newwords_custom:
    scores[w] = 1    

In [464]:
ltokenizer = LTokenizer(scores=scores)
for corp in jcorpus.corpus['asics']:
    print([tok for tok in ltokenizer.tokenize(corp, tolerance=0.0) if tok not in stopwords])
    #print([tok for tok in ltokenizer.tokenize(corp, tolerance=0.0) if tok in scores]) # 요걸쓰면 안된다. 너무 많이 없어져.
    #print('-------------------------------')

['족적', 'jpg', 'asics']
['12월', '둘째', '주', '이벤트', '스포츠', '브랜드', '방한', '대전', '겨울', '요거', '하나만', '딱', '있으면', '엄청', '따수운디', '안', '볼텨', '아식스', '뉴발란스', '르꼬끄', '나이키', '데상트', '리복', '패딩', '다운', '지금', '만나보세용', '퐁당']
['세일', '퐁당', '오', '포도몰', '이런', '세일', '혼자', '알고', '있기', '아까운', '유명', '브랜드', 'sale', '소식', '알려드립니다', '아식스', '레스모아', '나이키', '숲', '미샤']
['박근혜', '깜짝', '시구', '글러브', '태극마크', '운동화', '일제', '아식스', '이투데이', '아베오빠', '날좀보소', '날좀보소']
['박근혜', '깜짝', '시구', '글러브', '태극마크', '운동화', '일제', '아식스', '이투데이', '이럴', '알았다', '팬티', '에는', '욱일승천기', '들었을껄']
['무개념', '여인네', '에게', '국산운동화', '신기', '바라는', '것은', '정말', '지나친', '기대', '잖아요', '박근혜', '깜짝', '시구', '글러브', '태극마크', '운동화', '일제', '아식스', '이투데이']
['선동도', '가지가지한다', '김대중', '의', '아식스', '괜찮고', '박근혜', '의', '아식스', '문제', '란', '말인가', '박근혜대통령', '시구', '운동화', '일제', '아식스', '사실', '인가요', '박근혜', '깜짝', '시구', '글러브', '태극마크', '운동화', '일제', '아식스']
['박근혜대통령', '시구', '운동화', '일제', '아식스', '사실', '인가요', '박근혜', '깜짝', '시구', '글러브', '태극마크', '운동화', '일제', '아식스', '이투데이']
['박근혜', '깜짝', '시구', '글러브', '태극마크', '운

['착화감', '최고', 'asics', 'gelkayano23', '아식스', '젤카야노23', '러닝', '화', '런닝화', '헬스', '운동화', '운동', '슈즈', '운동', '하는남자', '신발', 'run']
['팔당대교', '아래', '물씬', '풍기는', '가을냄새', '오래된', 'red렌즈', 'fire렌즈로', '교체', '가을', '단풍', '런닝', '15km', 'nike', 'running', 'mizuno', 'asics']
['제', '족은', '소중하니까', '양말', '까지', '질러버렸네', 'asics', 'harajuku', 'japan', '지름', '쇼핑', 'shopping', '아식스', 'kayano23', 'asics']
['체열반사', '기능', '한층', '더', '업그레이드', '된', 'asics', 'thermo', 'down', 'jacket', '발열', '안감', '및', '덕다운', '충전재', '부터', '한기를', '차단', '해주는', '소매', '밑단', '목둘레', '디테일', '보온', '성을', '극대화', '하였습니다', '아식스', '다운자켓']
['event', 'asics', 'thermo', 'down', 'jacket', 'laun', 'ching', '체열반사', '기능', '한층', '더', '업그레이드', '된', '보온', '성', '주변', '제품', '출시', '소식', '널리', '알려주세요', '추첨', '통해', '체험', '기회', '드립니다']
['아베골프', '신상품', '던롭', '아식스', 'gel', 'ace', 'pro', '3', 'boa', '골프화', '4종', '출시', '되었어요', '구경', '오세요', '상품', '자세', '히', '보기', '아베골프', 'avegolf', 'golf', '골프화', 'golfswing', '아식스', 'asics', '핫템', '득템', '편한골프화']
['아베골프', '신상품', '던롭',

['지금', '아식스', '열풍', '키코', '프라다', '조합', '구레잇', 'now', 'korea', 'hottest', 'shoes', 'asics', 'kiko', 'with', 'prada', 'ding', 'asics', '아식스', '키코']
['러블리', '요정', '레깅스', '여신', '아식스', '이성', '경']
['스타일', '충전', '이성', '경', '세련미', '넘치', '모던', '애슬레저', '룩', '시선강탈', '이성', '경', '아식스']
['키코', '맛', '아식스', '모음', '젤', '님버스', '젤', '벤쳐', '졸트', '가성비', '갑', '갓식스', 'kiko', 'x', 'asics', 'look', 'gel', 'nimbus20', 'gel', 'venture', 'jolt', '아식스', '갓식스']
['아식스', '어글리', '슈즈', '젤', '버즈', '1', '15분', '만에', '완판', '아식스', '아식스', '젤버즈1', '어글리슈즈']
['이런', '선셋', '집근처', '볼수', '있다는건', '축복', '이지', '아식스', '카야노23', 'asics', 'kayano23', 'nrc', 'nikeplus', 'eveningrun', 'run']
['kiko', 'x', 'asics', 'collection', '키코', 'x', '아식스', '콜렉션', '2월', '6일', '분더샵', '판매', 'locobasic', '아식스', 'asics', 'kiko', 'kikokostadinov']
['kiko', 'x', 'asics', 'collection', '키코', '아식스', '콜렉션', '2월', '6일', '한정판매', '분더샵', 'locobasic', '아식스', 'supreme', 'sneakerhead']
['my', 'best', 'timing', 'for', 'running', '비기와서', '뛰다', '포기', '힘', '매우', '레어', '케

['신제품', '추천', '캐리어', '반팔티', '런닝화', '포도몰', '쌤소나이트', 'tbj', '아식스', '5', '15', '일', '까지', '10만원', '이상', '구매시', '구매금액', '별', '포도몰', '상품교환권', '증정']
['싸', '싸우', '자', '아식스']
['이번주', '일요일', '뚝섬', '하는', '안전공감마라톤', '배번', '기념티셔츠', '도착', '뚝섬유원지', '아식스']
['joggingtheworld', 'with', 'asics', 'underarmour', 'nike', 'diet', '다이어트', '언제', '살빼지', '이런', '설정', '샷', '정도', '푸훗', '서리', '풀공원']
['6pm', '크록스', '50', '리복', '아식스', '운동복', '및', '운동화', '70', '내추럴', '라이져', '버켄스탁', '샌달', '70']
['6pm', '코치', '60', '아식스', '70', '나인', '웨스트', '제시카', '심슨', '여성화', '50', '이하', '세일']
['케이스', '샀다', '이것', '도', '뭔가', '불안', '한', '모양', '이지만', '그냥', '써야지', '맘에', '들어', '문구', '도', '맘대로', '할', '있다', '아식스', 'bomsinal']
['스타일뉴스', '한그', '루', '상큼', '발랄', '캐주얼', '룩', '한그', '루', '아식스', '팬사인회', '아식스타이거', '스타일뉴스']
['asics', 'baseball', '사회', '인야구', 'joycover', '포수때', '사용', '토요일', '정비시간']
['아식스', '와', '12명', '의', '아식스', '플레이어', '의', '모든분들', '함께', '응원', '해주세요']
['6pm', '아식스', '60', '스케쳐스', '60', '클락', '스', '여성', '캐주얼화', '75', 'mia', '메든', '걸', 

['아식스쿨런', '쿨런', '쿨한', '러너들', '위한', '쿨', '러닝', '2015', 'asics', 'cool', 'run', '아식스', '아식스쿨런', '쿨런', '쿨러닝']
['아식스쿨런', '쿨런', '쿨한', '러너들', '위한', '쿨', '러닝', '2015', 'asics', 'cool', 'run', '아식스', '아식스쿨런', '쿨런', '쿨러닝']
['아식스쿨런', '쿨런', '쿨한', '러너들', '위한', '쿨', '러닝', '2015', 'asics', 'cool', 'run', '아식스', '아식스쿨런', '쿨런', '쿨러닝']
['아식스쿨런', '쿨런', '쿨한', '러너들', '위한', '쿨', '러닝', '2015', 'asics', 'cool', 'run', '아식스', '아식스쿨런', '쿨런', '쿨러닝']
['2015', 'asics', 'cool', 'run', 'master', '30', 's', '1080p', '님이', '공유', '아식스', '아식스쿨런', '쿨런', '쿨러닝']
['오글', '거리', '멘트', '와', '함께', '일단', '참가신청', '선발', '될런지는', '아직', '미지수', '런스타그램', '러닝', '초보', '아식스쿨런', '아식스', '쿨런', 'asics', 'coolrun']
['아식스쿨런', '쿨런', '쿨한', '러너들', '위한', '쿨', '러닝', '2015', 'asics', 'cool', 'run', '아식스', '아식스쿨런', '쿨런', '쿨러닝']
['onitsuka', 'tiger', 'mexico66', '아식스', '_', '마누라', '신발', '사주었다', '고맙다', '끝']
['새신', '신고', '뛰어보자', '폴짝', '아식스', 'g1', '캔디슈', '민트', '한달전', '구입', '해서', '이제서야', '개봉', '여름신발', '역시', '시원', '하구나', '민트색', '첨', '신어봐서', '고를때', '많이']
['

...

['아식스', '와', '함께', '수험생', '모두', '응원', '해주세요']
['혜박님', 'asics', '헤어밴드', '하고운동하니까편하당', '아식스']
['asics', '헤어밴드', '하고운동하니까편하당', '아식스']
['막', '어쩌고', '하다보니', '키보드', '랙이', '없어졌네', '이거', '뭐', '문제', '있어야', '센터', '가지', '아식스']
['동래', '구투룸', 'b', '피파13시즌베스트', 'j', '지지', '오토', '지성주지훈', '리버풀수아레스', '아식스', '이번']
['아식스', '신고', '뛰어보자', '앙큼', '올', '겨울', '러닝', 'g1', '과', '함께']
['6pm', '베테랑', '데이', '22', '균일가세일', '아식스', '60', '여성', '의류', '세일', '핸드백', '및', '악세서리', '75']
['아식스', '카메라', '쩌넹']
['아식스', '사진', '잘나오넹']
['다만널사랑하고있어', 'u', '로또사이트', 'u', '프로포즈', '영상제작', '트리플쉐이드', '재활용', '아식스', '청주', '아파트매매', '있습니다']
['11월', '둘째주', '패션', '관이벤트', '나이키', '비롯', '한', '스포츠브랜드', '캐주얼브랜드들', '까지', '확인', '해보세요', '플러스', '에스큐', '라임', '하임', '지오다노', '어스앤뎀', 'nii', 'hum', '스케쳐스', '레스모아', '나이키', '아식스', '리복']
['아식스', '페이스북', '이벤트', '아식스', '구스', '다운', '커플', '구스', '다운', '선물', '하기', '올', '겨울', '따스함', '선물', '싶은', '사람', '이유', '적어주세']
['아식스', '페이스북', '통해서', '겨울', '따스함', '함께', '나눌', '친구분', '초대', '해주세요']
['헐', '오늘', '본의아니게', '일탈', '깔맞춤', '못해

['이슈퐁당', '어제의', '대학생', '쇼핑인기검색어', '바로바로', '아식스', 'g1', '사진', '컬러', '말고도', '다양', '한', '컬러', '있어요', '5층', '매장', '구경', '신어보시고', '쇼핑', '하세요', '사진', '아식스', '공식홈']
['아식스', '운동화', '이렇게', '예쁘다구', '영업', '아식스', '젤라이트']
['asics', '19', 'ss', '캠페인', '2019', '02', '08', 'fri', '10', '00', 'am', '공개', '아식스', '나를움직여', 'theboyz', 'the_boyz', '더보이즈']
['2월8일', '오전', '10시', '아식스', '19', 'ss', '캠페인', '나를움직여', '공개', '더보이즈', '아식스']
['아식스', '키코', '밑창보강', 'asics', 'kiko']
['20', '20', '도쿄', '올림', '픽', '일본', '대표팀', '재활용', '유니폼', '입는다', '아식스', '20', '20도쿄올림픽일본대표팀유니폼']
['스티', '커', '제외', '더보이즈', '현재', '포카', '양도', '받아요', '즉입', '가능합니다', '가격', '제시', '해서', '디엠', '주세요', '기디업', '바인더', '스탬프', '소년', '공방', '스노우', '인생', '네컷', '노에어', '쇼케', '개인', '현재활', '오프', '에어', '아식스', '포카', '양도', '더보이즈', '덥즈', 'theboyz', '현재', '이재', '현']
['항녀니', '검정색', '빨간색', '두개', '신었구나', '아식스', 'asics']
['아식스', '탕진', '하라고', '인센티브', '나오나보다', '더보이즈', 'theboyz', '주학년', '학년', 'juhaknyeon', '아식스', 'asics']
['더보이즈', '아식스', '엽서', '포카', '1장', '2', '000', 'won'

['노익스큐즈', '화이팅', '가민', '에프터샥', 'slrc', '멍뭉이', '아미', '노바이탈', '아식스', '차크라', '메달', '고', '국민', '체육진흥공단', '스피돔', '토리', '브엔터테인먼트', '디자', '드', '전설', '의다방구', 'p', 's', '재밌는추억만들어주셔서감사합니다']
['노익스큐즈', '화이팅', '가민', '에프터샥', 'slrc', '멍뭉이', '아미', '노바이탈', '아식스', '차크라', '메달', '고', '국민', '체육진흥공단', '스피돔', '토리', '브엔터테인먼트', '디자', '드', '전설', '의다방구', 'p', 's', '재밌는추억만들어주셔서감사합니다']
['asics', '와', 'dc', '의', '엔도서로', '활약', '듸제이불화자씨', '젊음', '의', '베라', '방문', '한다는', '마음', '한껏', '멋을', '내보았습니다', '패딩', '꺼내입고', 'gel', 'pt', 'g를', '신고가서', '밟혀', '서', '까매짐', 'asics', 'asics', 'tiger', 'dc', 'dcshoes', 'dc', 'shoecousa', 'dj', 'djlife', 'davotab', 'vera']
['아식스', '스페셜', '포카', '양도', '빋아입니다', '디엠주세요', '더보이즈', 'theboyz', '상연', '영훈', '제이콥', '현재', '주연', '케빈', '큐', '뉴', '양도', '아식스', '포카']
['일본', '의', '스포츠', '브랜드', '아식스', '의', '브랜드', '중', '하나인', '오니츠카', '타이거', 'onitsuka', 'tiger', '2019년', '2월', '새로운', '프리미엄', '브랜드', '인', 'the', 'onitsuka', '디', '오니츠카', '런칭', '합니다', '오니츠카타이거', '아식스', 'theonitsuka', '디오', '니츠카', '더오니츠카', '스니커', '일

['지난', '주말', '노박', '조코비치', '의', '두', '게임', '보며', '더욱', '가지고', '싶어졌다', '아직', '우리나라', '에는', '출시', '하지', '않은', '15', '0', '짜리', '빨간색', '아식스', 'gel', 'resolution', '7', 'no', 'vak', '아식스', '노박조코비치', 'no', 'vakdjokovic', 'gelresolution7novak', 'gelresolution']
['더보이즈', '부산', '아식스', '일일스태프', '동행', '구해요', '9시', '10시', '사이', '도착', '가능', '해요', '편하게', '디엠주세요', '더보이즈', '아식스', '동행']
['더보이즈', '아식스', '스페셜', '포토카드', '각', '준등포', '0', '4', '영훈', '큐', '제외', '전', '멤버', '가능', '당첨', '인증', '가능', '해요', '더보이즈', '아식스']
['18', '1020', '더보이즈', '아식스', '광복직영점', '양도합니다', '디엠주세요', '더보이즈', '아식스', '부산']
['더보이즈', '아식스', '스페셜', '포토카드', '각', '준등포', '0', '4', '영훈', '제외', '전', '멤버', '가능', '당첨', '인증', '가능', '해요', '더보이즈', '아식스']
['더보이즈', '아식스', '스페셜', '포토카드', '각', '준등포', '0', '5에', '양도합니다', '제발', '받아', '주세요', '많이', '받으시면', '에눌', '해', '드려', '더보이즈', '아식스']
['p', 'i', 'c', 'k', 'm', 'e', 'u', 'p', 'iphone', 'x', 'bethebestversionofme', 'zarabag', 'asics', 'hm', 'thailand', 'fashion', 'welcometotherealworld', 'seoul', 'gangnam',

['더워보', '이지만', '부럽', '아식스', '카야노23', 'asics', 'kayano23', 'nrc', 'nikeplus', 'run', 'runner', 'running', 'run', 'stagram', 'bedokreservoir', 'bedok', 'sing', 'apore', '달리기', '베독호숫', '베독', '싱가포르', '런스타그램', 'mara', 'thon']
['간만', '좀', '뛰었군', '아식스', '카야노23', 'asics', 'kayano23', 'nrc', 'nikeplus', 'run', 'runner', 'running', 'run', 'stagram', 'bedokreservoir', 'bedok', 'sing', 'apore', '달리기', '베독호숫', '베독', '싱가포르', '런스타그램', 'mara', 'thon']
['아식스', '강남', '매장', '더보이즈', '쇼', '윈도우', '더보이즈', 'the_boyz', '아식스']
['the', 'boyz', 'x', 'asics', 'ugly', '똑같은', '건', '지루해', '또', '다른', '나를', '보여줄', '거야', 'get', 'ugly', 'get', 'jolt']
['ugly', '똑같은', '건', '지루해', '또', '다른', '나를', '보여줄', '거야', 'get', 'ugly', 'get', 'jolt', '아식스', 'the_boyz', '더보이즈', '뉴', '주학년']
['ugly', '못생긴', '게', '아니야', '개성', '거지', 'get', 'ugly', 'get', 'jolt', '아식스', 'the_boyz', '더보이즈', '에릭']
['아베골프', '신상품', '던롭', '아식스', 'gel', 'ace', 'pro', '3', 'boa', '골프화', '4종', '출시', '되었어요', '구경', '오세요', '상품', '자세', '히', '보기', '아베골프', 'avegolf', 'g

In [415]:
class JsonCorpus:
    def __init__(self, *fnames, textkey='text'):
        self.fnames = fnames
        self.textkey = textkey
        self.corpus = self._corpus()
            
    def _corpus(self):
        corpus = {}
        for fname in tqdm_notebook(self.fnames):
            with open(fname, encoding='UTF-8-sig') as f:
                js = json.load(f)
                topitem = next(iter(js))
                sents = []
                
                for doc in js[topitem]:
                    sents.append(preproc(doc[self.textkey]))
                    
                corpus[topitem] = sents
                
        return corpus
                
        
    def __iter__(self):
        for sents in self.corpus.values():
            yield from sents
        
        #for fname in self.fnames:
        #    with open(fname, encoding='UTF-8-sig') as f:
        #        js = json.load(f)
        #        topitem = next(iter(js))
        #        for doc in js[topitem]:
        #            yield doc[self.textkey]
                    
    def __len__(self):
        return sum([len(sents) for sents in self.corpus.values()])
        
        #lens = []
        #for fname in self.fnames:
        #    with open(fname, encoding='UTF-8-sig') as f:
        #        js = json.load(f)
        #        topitem = next(iter(js))
        #        lens.append(len(js[topitem]))
        #return sum(lens)

In [416]:
#fnames = ['scrapped/twitter/asics (2010.01.01-2019.01.31).json','scrapped/twitter/crocs (2010.01.01-2019.01.31).json']
fnames = ['scrapped/twitter/' + fname for fname in os.listdir('scrapped/twitter')]#[:5]
jcorpus = JsonCorpus(*fnames)

In [418]:
#jcorpus.corpus

In [429]:
ltokenizer = LTokenizer(scores=scores)
for corp in jcorpus.corpus['asics']:
    print([tok for tok in ltokenizer.tokenize(corp, tolerance=0.0) if tok in scores])

['족적', 'asics']
['12월', '둘째', '이벤', '스포', '브랜드', '방한', '대전', '겨울', '요거', '하나만', '딱', '엄청', '안', '아식스', '뉴발란스', '르꼬끄', '나이키', '데상트', '리복', '패딩', '지금', '퐁당']
['세일', '퐁당', '포도몰', '세일', '혼자', '알고', '있기', '유명', '브랜드', 'sale', '소식', '아식스', '레스모아', '나이키', '숲', '등']
['박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스', '아베오빠', '날좀보소', '날좀보소']
['박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스', '내', '줄', '팬티']
['여인네', '국산운동화', '신기', '정말', '기대', '박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스', 'vi', 'a']
['김대중', '아식스', '박근혜', '아식스', '문제', '말인가', '박근혜대통령', '시구', '운동화', '일제', '아식스', '데', '사실', '박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스']
['박근혜대통령', '시구', '운동화', '일제', '아식스', '데', '사실', '박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스']
['박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스']
['박근혜', '깜짝', '시구', '글러브', '엔', '태극마크', '운동화', '일제', '아식스', '대통령', '홍보', '자리']
['오늘', '햄버거', 'asics']
['안돼', '용기', '체리핑크', '눈만', '체리', '핑', 'asics', 'g1', '하지원',

['아식스', 'asics', '키코', '코스', '2019', 's', 's', '협업', '컬렉션', '스트릿패션', '스트릿', '패션', '스트릿브랜드', '브랜드', '브랜드컬렉션', '컬렉션', '패션매거진', '매거진', '스트릿컬처', '서브컬처', '아식스', 'asics', '키코']
['떠오르', '대세', 'nike', 'adidas', 'asics']
['lee', '인스타그램', '더보이즈', '함께', '시간', '올드보', '에너지', '소중', '시간', 'good', '더보이즈', '아식스', 'asics']
['boy', '아식스', 'asics', '스트리트웨어', '컬렉션', '스트릿패션', '스트릿', '패션', '스트릿브랜드', '브랜드', '브랜드컬렉션', '컬렉션', '패션매거진', '매거진', '스트릿컬처', '서브컬처', 'boy', '아식스', 'asics', '프랑스', '스포츠웨어', '스트릿웨어']
['180', 'asicskr', '당신', '산뜻한', '봄날', '느낌', '스타일리쉬함', 'knit', '잊지', '마세', 'asics', '아식스', 'lee', '이성', '이성', 'lee']
['180', 'asicskr', '편안', '착용감', '스타일리쉬', '디자', '언제', '어디서', '당신', '빛나게', '줄', '선물', '특별함', '지금', 'asics', '아식스', 'lee', '이성', '이성', 'lee']
['체력검정', '의외', '기록', '선물', '운동화', '기록', '영향', '아식스', '타사']
['금요일', '업데이트', '노드', '처리', '개선', '노드', '사용', '코드', '제거', '노드', '추적', '개선', '됨', '알고', '개선', '일치', 'as', '저항', '암호화폐', '블록', '업데이트', 'cr', 'asics', 'as']
['베독호숫', '언제나', '옳다', '아식스', 'asics', 'nrc', 'n

['아식스', '60', '스케쳐스', '60', '클락', '스', '등', '여성', '캐주얼화', '75', 'mia', '70', 'dc', 'rox', 'y', '등', '스케이트', '화', '70']
['노스페이스', '세일', '스튜어트', '세일', '아식스', '60', '세일']
['or', 'k', 'miss', 'don', 'zooyork', 'asics', '여의도한강시민공원']
['상품권', '선물', '밥', '선물', '상품권', '아식스', 'asics', '한국체육대학교']
['오니츠카타이거', '아식스', 'onitsukatiger', '선물', '운동화', '실버', '선물고마워', '잘신을게']
['스타일', 'm', '페이스북', '좋아', '공유', '아식스', '워킹화']
['나의', '달리기', '친구', '아식스', 'new', 'running', 'asics']
['축구화', '정도', '거', '아닌가', '여러분', '신발장', '모습', '신발장', '자랑질', '푸마', '아디다스', '나이키', '미즈노', '아식스', 'puma', 'adidas', 'nike']
['오늘', '운동화', 'asics']
['리그', '바람', 'asics', 'si', 'shoe', '잔뽕', '잔뽕', '잔뽕', '너', '사회', '마스킹', '생명']
['asics', '손', '얻는', '또', '기회', 'y', '업', 'asics', 'asics', 'asics', 'asics']
['아식스', '봄', '꽃', '잔디']
['나랑', '수고', '운동화', '원래', '버리고', '했는', '데', '버릴수', '신을까', 'asics']
['아식스', '아식스', '프로모션', '2015', '사이판', '마라톤', '여러분', '아식스', '러닝', '신고', '사이판', '마라톤', '참가', '사이판', '명소', '누구', '여행']
['20150218', '화분', '아식스', '화분', '

['다리스타그램', '새옷', '스타그램', '아식스', '운동화', '검정운동화', '득템', '청바지', '찢청', '득템', '득템', '엄카', '위대함', '그래']
['나의', '스포', '실행', '신발', '아식스', 'asics']
['run', 'adidas', 'un', '인수', 'run', 'asics', '인수', '발표', 'fmcg', 'it', '결합', '이상적인', '모델']
['수요', '일', '저녁', '아식스', '달리기', '잠수교', '왕복', '러닝크루', '최종', '선정', '열심히', '건강', '다이어트', '마라톤', 'run', 'running', '운동']
['프랑스', '이탈리아', '스위스', '3개', '달리기', '아식스']
['추억', '브랜드', '아식스', '케이스위스', '뉴니', '언니', '캠퍼스', '캠퍼스', '홈페이지', '직진']
['추억', '브랜드', '아식스', '케이스위스', '뉴니', '언니', '캠퍼스', '캠퍼스', '홈페이지', '직진']
['토요일', '오전', '달리기', '즐겁게', '사람들', '함께', '아식스', 'run', 'running', '마라톤', '운동', '다이어트']
['와이프', '직구', '오니즈카타이거', '35', '싸다', '직구', '오니즈카타이거', '아식스']
['수요', '일', '달리기', '아식스', '아식스', '딱', '뛰기', '날씨', 'run', '마라톤', '다이어트', '운동', '건강', '잠수교']
['2015', '12', '19', '토', '오전', '아식스', '아식스', '달리기', '마라톤', '준비', 'run', '건강', '운동', '다이어트', 'asics']
['백수넷', '아식스', 'gt', '러닝', '화', '32', '000', '등록', '백수넷', '아식스', 'gt', '러닝', '화', '32', '000']
['공개', '역시', 'asics', 'onitsukati

['11월', '패션', '나이키', '비롯', '스포츠브랜드', '캐주얼브랜드들', '확인', '플러스', '라임', '지오다노', '어스앤뎀', 'nii', 'hum', '스케쳐스', '레스모아', '나이키', '아식스', '리복']
['아식스', '페이스북', '이벤', '아식스', '구스', '커플', '구스', '선물', '겨울', '따스함', '선물', '사람', '이유', '적어주세']
['아식스', '페이스북', '통해서', '겨울', '따스함', '함께', '친구분', '초대']
['오늘', '일탈', '깔맞춤', 'asics']
['아식스', '축구화', '테스트', '클래식', '일본', '남포동', '요즘', '클래식', '축구화', '날', '테스트', '첫번째', '버젼']
['아식스', '깜짝이벤트', 'sns어워드', '대상', '수상', '아식스', '페이스북', '2014', 'sns', 'award', '스포츠부문', '대상']
['아식스', '2014', 'sns', 'award', '스포츠부문', '대상', '수상', '함께', '축하']
['lee', 'asics', 'korea', '2014', 'f', 'w', '아식스', 'g1', '베리베리', 'tvc', '메이킹필름']
['lee', 'asics', 'korea', 'bts', '이종석']
['관계자', '선물', 'kbo', '삼성', '라이온즈', '지급', '품', 'asics', '야구', '사회']
['2014', 'f', 'w', '아식스', 'g1', '베리베리', 'tvc', '메이킹필름', '이종석', 'asics', 'bts']
['2014', 'f', 'w', '아식스', 'g1', '베리베리', 'tvc', '메이킹필름', '이종석', 'asics', 'bts']
['아식스', '행사', '롯데', '백화점', '한가', '아식스', '정렬']
['목요일', '저녁식사', '소주', '함께', '츄리닝', '함께', 'co', 'suprem

['더보이즈', '양도', '공식', '포카', '굿즈', '판매합니다', '1', '1장1500', '2장', '250', '0', '1장', '3장', '55', '00', '모든', '종류', '개봉', '앨범', '리스트', '업데이트', '12', '17', '주연', '선우', '학년', '제이콥', '뉴', '아식스', '스쿨룩', '스']
['겨울', '워커', '부츠', '신다', '작년', '부츠', '고마움', '후보', '찾고', '있는데', '마음', '아이들', '패딩', '부츠', '부츠', '부츠', '방한', '화', 'asics']
['모닝', '언제나', '옳다', '아식스', 'asics', 'nrc', 'nikeplus', 'run', 'runner', 'running', 'run', 'sing', '달리기', '베독호숫', '싱가포르', 'mara']
['더보이즈', '양도', '공식', '포카', '굿즈', '판매합니다', '1', '1장1500', '2장', '250', '0', '1장', '3장', '55', '00', '모든', '종류', '개봉', '앨범', '리스트', '업데이트', '12', '15', '주연', '선우', '학년', '제이콥', '뉴', '아식스', '스쿨룩', '스']
['더보이즈', '양도', '공식', '포카', '굿즈', '판매합니다', '1', '1장1500', '2장', '250', '0', '1장', '3장', '55', '00', '모든', '종류', '개봉', '앨범', '리스트', '업데이트', '주연', '선우', '학년', '제이콥', '뉴', '아식스', '스쿨룩', '스']
['신상', '패션', '텅장', '커플롱패딩', '워치', '커플', '롱패딩', '씨', '지이크파렌하이트', '콜라보레이션', '컬래버레이션', '아식스', '스타일', '패셔니스타', '패션피플', '오늘', '신상', '12월', '텅장', '털이', '너', '커플', '롱패딩', '시

['더보이즈', '아식스', '스페셜', '포토카드', '0', '제외', '멤버', '가능', '당첨', '인증', '가능', '더보이즈', '아식스']
['18', '더보이즈', '아식스', '광복직영점', '양도합니다', '더보이즈', '아식스', '부산']
['더보이즈', '아식스', '스페셜', '포토카드', '0', '제외', '멤버', '가능', '당첨', '인증', '가능', '더보이즈', '아식스']
['더보이즈', '아식스', '스페셜', '포토카드', '0', '5에', '양도합니다', '제발', '많이', '에눌', '드려', '더보이즈', '아식스']
['i', 'c', 'k', 'm', 'e', 'u', 'iphone', 'zarabag', 'asics', 'hm', 'fashion', 'seoul', '강남페스티벌']
['더보이즈', '아식스', '스페셜', '포토카드', '0', '5에', '분철합니다', '당첨자', '인증', '가능', '토요일', '월요일', '보내드려', '더보이즈', '아식스', '제이콥', '주연', '뉴', '선우']
['아식스', '스페셜', '포토카드', '제외', '분철', '0', '더보이즈', '아식스']
['아식스', '스페셜', '포토카드', '제외', '분철', '0', '더보이즈', '아식스']
['더보이즈', '아식스', '스페셜', '포카', '분철합니다', '모든멤버', '가능', '당첨', '문자', '인증', '가능', '멤버', '가격', '제시', '더보이즈', '더보이즈아식스', '더보이즈포카', '스페셜', '포카', '더보이즈아식스', '제이콥', '주연', '뉴', '선우', '아식스포카', '아식스']
['더보이즈', '아식스', '스페셜', '포카', '분철합니다', '모든멤버', '가능', '당첨', '문자', '인증', '가능', '멤버', '가격', '제시', '더보이즈', '더보이즈아식스', '더보이즈포카', '스페셜', '포카', '더보이즈아식스', '제이

['아베골프', '신상품', '던롭', '아식스', 'pro', '골프화', '4종', '출시', '구경', '상품', '자세', '히', '보기', '아베골프', '골프화', '아식스', 'asics', '득템']
['슈즈', '열풍', '계속', '발렌시아가', '루이비통', '휠라', '아식스', '어글리슈즈', '휠라', '아식스', '발렌시아가', '루이비통', '구찌']


In [419]:
noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(jcorpus)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 493329 from 429969 sents. mem=4.087 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=6680411, mem=3.168 Gb
[Noun Extractor] batch prediction was completed for 112092 words
[Noun Extractor] checked compounds. discovered 106245 compounds
[Noun Extractor] postprocessing detaching_features : 128251 -> 117025
[Noun Extractor] postprocessing ignore_features : 117025 -> 116918
[Noun Extractor] postprocessing ignore_NJ : 116918 -> 116458
[Noun Extractor] 116458 nouns (106245 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=3.243 Gb                    
[Noun Extractor] 75.49 % eojeols are covered


In [420]:
scores = {noun:nounscore.score for noun, nounscore in nouns.items()}
for kw in newwords_kw:
    scores[kw] = 1

In [421]:
jcorpus.corpus['asics']

['족적 jpg asics',
 '12월 둘째주 이벤트 스포츠 브랜드 방한대전 겨울에 요거 하나만 딱 있으면 엄청 따수운디 안 볼텨 아식스 뉴발란스 르꼬끄 나이키 데상트 리복 패딩 다운 지금 만나보세용 퐁당',
 '세일 퐁당 오 포도몰에서 이런 세일이 혼자 알고 있기 아까운 유명 브랜드 sale 소식 알려드립니다 아식스 레스모아 나이키 숲 미샤 등',
 '박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스 이투데이 에서 아베오빠 날좀보소 날좀보소',
 '박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스 이투데이 에서 내 이럴 줄 알았다 팬티에는 욱일승천기 들었을껄',
 '이 무개념 여인네에게 국산운동화 신기를 바라는 것은 정말 지나친 기대잖아요 박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스 이투데이 via',
 '선동도 가지가지한다 김대중의 아식스는 괜찮고 박근혜의 아식스는 문제란 말인가 박근혜대통령 시구시 운동화 일제 아식스라는 데 사실인가요 박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스',
 '박근혜대통령 시구시 운동화 일제 아식스라는 데 사실인가요 박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스 이투데이 에서',
 '박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스 이투데이 에서',
 '박근혜 깜짝 시구 글러브엔 태극마크 운동화는 일제 아식스 이투데이 에서 대통령 보좌진은 뭐해 홍보수석은 뭐하는 자리야',
 '오늘은 햄버거 monkids burger lettuce asics gelyte3',
 '아무나 안돼 하지만 난돼 용기를 내는거야 체리핑크 눈만 맞아도 입만열어도 웃지마 정들어 체리핑 asics g1 하지원 존예 언제나이상형',
 'asics gtcool 2013octoberreleases shose 아식스 지티쿨 운동화 신발 2hchic 색상 깔끔하네 우선 가격이나 알아봐야지',
 '시작 공연 seoul10k asics',
 '짐 여기에 있음 soeoul10k asics',
 'asics 2013 f

In [414]:
with open('scrapped/twitter/asics (2010.01.01-2019.01.31).json', encoding='UTF-8-sig') as f:
    jsn = json.load(f)
    
preproc(jsn['asics'][28]['text'])

'대우의 달콤한 asics wet swimsuit lesbians from westland mi rss1sexeaparis 177'

In [383]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = '((?: rt)|(?:^rt))[^ @]?'
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    SPECIALS = r'([^ a-zA-Z0-9_\u3131-\u3163\uac00-\ud7a3]+)|([ㄱ-ㅣ]+)'
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    #PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    #out = {}
    #text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), '', text.lower())
    text = text.lower()
    
    if remove_url:
        text = re.sub(URL, ' ', text)

    if remove_mention:
        text = re.sub(MENTION, ' ', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, ' ', text)
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    #set_trace()
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), ' ', text)
    return re.sub(LONG_BLANK, ' ', text).strip()